In [1]:
# Loading of libraries
import numpy as np
import scipy.spatial.distance
import matplotlib.pyplot as plt
from numba import jit
import time
import warnings

%matplotlib inline

In [2]:
# Design variables
AverageRadius            = 1
DeviationRadius          = AverageRadius / 10
ParticleCount            = 500
Viscosity                = 1E3
ViscosityR               = 32 / 3 * Viscosity
RotationalViscoisty      = 1E3
RotationalViscoistyR     = 4 * 3.1415 * RotationalViscoisty
NeighbourDistance        = 2.7 * AverageRadius
SpringConstant           = 1
Chi                      = 1         # Rotational viscosity    - 1 in natural units
Zeta                     = 1         # Translational visocsity - 1 in natural units
k                        = 1         # Sprint constant         - 1 in natural units

# Simulation variables
NeighbourCutoff          = 2.7
TimeStep                 = 1
L_align                  = 0.1       # Range from 0.1 to 1
L_noise                   = 0.04      # Range from 0.04 to 0.08
L_F_in                   = 0.3
L_T_in                   = 3
L_self                   = 0.03
T_align                  = L_align * k * Chi / Zeta
T_noise                  = np.sqrt(L_noise / Zeta / TimeStep * 2 * k * Chi**2)
F_in                     = L_F_in * AverageRadius * k
T_in                     = L_T_in * Chi * k / Zeta
F_self                   = L_self * k * AverageRadius


# Initiate the pengiuins
Radius                   = np.random.normal(AverageRadius, DeviationRadius, ParticleCount)
Location                 = np.array(np.vstack((
                             np.tile(np.arange(0,10), ParticleCount / 10) * 2,
                             np.repeat(np.arange(0,ParticleCount / 10), 10) * 2
                           )), dtype=np.float64).T# + np.random.normal(0,1,(ParticleCount,2))
Velocity                 = np.zeros((ParticleCount, 2))
Orientation              = np.ones(ParticleCount) * 3.1415 / 2 + np.random.uniform(-np.pi/4, np.pi/4, ParticleCount)
AngularVelocity          = np.zeros(ParticleCount)


In [3]:
def ParticleForce(Orientation, OnBoundary, ThetaOut, Overlap, F_self, F_in, k):
    # Particle force = Self-Propulsion + Boundary + Repulsion
    
    Dir =  np.array([np.cos(Orientation), np.sin(Orientation)]).T
    PropulsionForce = F_self * Dir
    BoundaryForce   = F_in * (ThetaOut[:,np.newaxis] - np.pi) * OnBoundary[:,np.newaxis] * Dir
    RepulsionForce  = k * Overlap;
    
    return PropulsionForce + BoundaryForce + RepulsionForce

def ParticleTorque(Orientation, OnBoundary, DeltaTheta, DeltaOrientations, T_in, T_noise, T_align):
    # Particletorque = Self-Torque + Noise + Align
    BoundaryTorque = T_in * DeltaTheta * OnBoundary
    NoiseTorque    = T_noise * np.random.uniform(-1,1,Orientation.shape[0])
    AlignTorque    = T_align * DeltaOrientations
    
    return BoundaryTorque + NoiseTorque + AlignTorque

@jit(nopython=True)
def NeighbourAngles(Location, Orientation, Neighbours):
    n = Location.shape[0]
    ThetaOut = np.zeros((n))
    DeltaTheta = np.zeros((n))
    DeltaOrientations = np.zeros((n))
    for i in range(n):
        # select neighbours
        CurrentNeighbours = np.where(Neighbours[i,:])[0]
        NumNeighbours = CurrentNeighbours.shape[0]
        
        if (NumNeighbours > 0):
            DeltaPsi = (Orientation[CurrentNeighbours] - Orientation[i])
            DeltaPsi = (DeltaPsi + np.pi) % (2*np.pi) - np.pi
            DeltaOrientations[i] = np.sum(DeltaPsi)
        
        if (NumNeighbours > 1):
            # calculate angles between neighbours and order them
            Offsets = Location[CurrentNeighbours,:] - Location[i,:]
            Angles = np.arctan2(Offsets[:,1], Offsets[:,0]) % (2*np.pi)
            SortedAngles = np.sort(Angles)
            
            # calculate angle differences between subsequent neighbours
            DiffAngles = np.zeros((NumNeighbours))
            # numba doesn't support roll unfortunately
            for j in range(NumNeighbours):
                DiffAngles[j] = ((SortedAngles[((j+1)%NumNeighbours)] - SortedAngles[j])) % (2*np.pi)
            
            # determine the largest gap in angles, which corresponds to theta-out
            ThetaOut[i] = np.max(DiffAngles)
            
            # calculate the absolute angle of the theta-in direction
            BoundaryNeighbour = np.argmax(DiffAngles)
            ThetaIn = (SortedAngles[BoundaryNeighbour] - 0.5 * (2*np.pi - ThetaOut[i])) % (2*np.pi)
            
            # calculate signed angle difference between orientation and theta-in
            DeltaTheta[i] = (ThetaIn - Orientation[i])
            DeltaTheta[i] = (DeltaTheta[i] + np.pi) % (2*np.pi) - np.pi
    return ThetaOut, DeltaTheta, DeltaOrientations

@jit(nopython=True)
def NeighbourOverlap(Location, Neighbours, Radius, Distances):
    Overlap = np.zeros((Location.shape[0],2))
    for i in range(Location.shape[0]):
        CurrentNeighbours = np.where(Neighbours[i,:])[0]
        NumNeighbours = CurrentNeighbours.shape[0]
        if (NumNeighbours == 0):
            continue
        for j in CurrentNeighbours:
            Dir = Location[i,:] - Location[j, :]
            d = Distances[i,j]
            r = Radius[j]
            R = Radius[i]
            # we use a simplified 1st order approximation for the overlap
            # by approximating the arcs by straight lines
            c = 2*d*r - d**2
            if (c < 0):
                continue;
            A = ((r+R)-d) * (2*np.sqrt(c)) / 2;
            Overlap[i,:] += A / d * Dir
        

In [4]:
max_steps = np.int(1E4);
#max_steps = 0;

warnings.simplefilter("error")

Torque               = np.zeros((ParticleCount))
Force                = np.zeros((ParticleCount,2))
Overlap              = np.zeros((ParticleCount,2))
ThetaOut             = np.zeros((ParticleCount))
DeltaTheta           = np.zeros((ParticleCount))
DeltaOrientations    = np.zeros((ParticleCount))
Distances            = np.zeros((ParticleCount, ParticleCount))
Neighbours           = np.zeros((ParticleCount, ParticleCount), dtype=np.bool)
OnBoundary           = np.zeros((ParticleCount))

start = time.clock();
for step in range(0,max_steps):
    if (step%100 == 0):
        print(step)
    
    Distances[:,:] = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(Location))
    Neighbours[:] = (Distances < NeighbourCutoff) & ~np.eye(Location.shape[0], dtype=np.bool)
    ThetaOut[:], DeltaTheta[:], DeltaOrientations[:] = NeighbourAngles(Location, Orientation, Neighbours)
    OnBoundary[:] = (ThetaOut > np.pi)
    
    Overlap[:,:] = NeighbourOverlap(Location, Neighbours, Radius, Distances)
    
    Torque[:]          = ParticleTorque(Orientation, OnBoundary, DeltaTheta, DeltaOrientations, T_in, T_noise, T_align)
    AngularVelocity[:] = Torque / (Chi * Radius*Radius)
    Orientation[:]     = Orientation[:] + TimeStep * AngularVelocity
    
    
    Force[:,:]         = ParticleForce(Orientation, OnBoundary, ThetaOut, Overlap, F_self, F_in, k)
    Velocity[:,:]      = Force / (Zeta * Radius[:,np.newaxis])
    Location[:,:]      = Location[:,:] + TimeStep * Velocity
    
print(time.clock() - start)

warnings.simplefilter('default')

0


RuntimeWarning: invalid value encountered in less

In [9]:
warnings.simplefilter('default')
print(Overlap)
step

[[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]
 [ nan  nan]

1

In [ ]:
## Visulaization
fig = plt.figure();
plt.rcParams['figure.figsize'] = (25,25)
ax = plt.gca();
for i in range(Location.shape[0]):
    circle = plt.Circle((Location[i,0], Location[i,1]), Radius[i], color='b', fill=False)
    ax.add_artist(circle);
    ax.arrow(
        Location[i,0],
        Location[i,1],
        0.5 * np.cos(Orientation[i]),
        0.5 * np.sin(Orientation[i]),
        head_width = 0.5,
        head_length = 0.5
    )
a = 1.5;
xd = np.mean(Location[:,0]) - np.abs(np.mean(Location[:,0]) - np.min(Location[:,0])) * a
xu = np.mean(Location[:,0]) + np.abs(np.mean(Location[:,0]) - np.max(Location[:,0])) * a
yd = np.mean(Location[:,1]) - np.abs(np.mean(Location[:,1]) - np.min(Location[:,1])) * a
yu = np.mean(Location[:,1]) + np.abs(np.mean(Location[:,1]) - np.max(Location[:,1])) * a

xu = max(xu, yu);
xd = min(xd, yd);
ax.set_xlim(xd, xu)
ax.set_ylim(xd, xu)
plt.show()

In [ ]:
print(np.mean(np.abs(Velocity)))